In [1]:
import numpy as np
import pandas as pd
import math as e
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import random
import math

class Node:
    def __init__(self,x,gradient,hessian,idxs,subsample_cols = 1, min_leaf = 1, min_child_weight = 1, 
                 depth = 2, lambda_ = 1, gamma = 1, eps = 0.01):
        self.x = x
        self.gradient = gradient
        self.hessian = hessian
        self.idxs = idxs
        self.subsample_cols = subsample_cols
        self.min_leaf = min_leaf
        self.min_child_weight = min_child_weight
        self.depth = depth
        self.lambda_ = lambda_
        self.gamma = gamma
        self.eps = eps
        self.rows_cnt = len(idxs)
        self.col_count = x.shape[1]
        
        # print(self.idxs)
        # print(gradient[self.idxs])
        # Select a subspace of attributes.
        self.column_subsample = np.random.permutation(self.col_count)[:round(self.subsample_cols*self.col_count)]
        
        self.val = self.calc_val(self.gradient[self.idxs],self.hessian[self.idxs])
        
        self.score = float('-inf')
        self.find_split()
        # print(self.is_leaf)
        
    def calc_val(self,gradient,hessian):
        # Calculate the optimal value of a leaf.
        value = np.sum(gradient)/(np.sum(hessian)+self.lambda_)
        value = value*-1
        return value

    def find_split(self):
        # Scans through every column & calculates the best split point. If node is split, 
        # two other nodes are created accordingly. If no split is better then we just return 
        # without making splits.
        
        for col in self.column_subsample:
            self.find_greedy_split(col)
        
        # for col in self.column_subsample:
        #     self.find_approx_split(col)
            
                
        if (self.is_leaf()):
            return
        
        split_col = self.split_col()
        
        lhs = np.nonzero(split_col <= self.split)[0]
        rhs = np.nonzero(split_col > self.split)[0]
        
        
        self.lhs = Node(x = self.x, gradient= self.gradient, hessian = self.hessian, idxs = self.idxs[lhs],
                        min_leaf= self.min_leaf, depth = self.depth - 1, lambda_= self.lambda_, 
                        min_child_weight = self.min_child_weight, eps = self.eps, 
                        subsample_cols = self.subsample_cols)
        
        self.rhs = Node(x = self.x, gradient= self.gradient, hessian = self.hessian, idxs = self.idxs[rhs],
                        min_leaf= self.min_leaf, depth = self.depth - 1, lambda_= self.lambda_, 
                        min_child_weight = self.min_child_weight, eps = self.eps, 
                        subsample_cols = self.subsample_cols)
    
    def find_greedy_split(self,col):
        # For the given attribute 'col' we try to calculate the best possible split.
        # Globally updates the best score and split point if a better split point is found.
        
        x = self.x.values[self.idxs, col]   # returns numpy array of values of 'col' at certain indices
        
        
        for r in range(0,self.rows_cnt):
            lhs = x <= x[r]
            rhs = x > x[r]
            
            lhs_indices = np.nonzero(x <= x[r])[0]            
            rhs_indices = np.nonzero(x > x[r])[0]
            
            if(len(lhs_indices) < self.min_leaf or len(rhs_indices) < self.min_leaf or
               self.hessian[self.idxs][lhs_indices].sum() < self.min_child_weight or 
               self.hessian[self.idxs][rhs_indices].sum() < self.min_child_weight):
                continue
            
            poss_score = self.gain(lhs_indices,rhs_indices)
            
            if poss_score > self.score:
                # print(col)
                self.col = col
                self.score = poss_score
                self.split = x[r]
    
    def find_approx_split(self, col):
        x = self.x.values[self.idxs, col]
        hessian_ = self.hessian[self.idxs]
        df = pd.DataFrame({'Feature': x, 'hess': hessian_, 'id': list(range(len(self.idxs)))})
        df.sort_values(by=['Feature'], ascending=True, inplace=True)
        df.reset_index(inplace=True)
        hess_sum = df['hess'].sum()
        df['rank'] = df.apply(lambda x: (sum(df[df['Feature'] <= x['Feature']]['hess'])) / hess_sum, axis=1)

        prev_idx = 0
        bins_done = 0
        prev_rank = 0
        total_bins = int(1 / self.eps)

        for i in range(0, len(df)):
            if (np.abs(df.loc[i, 'rank'] - prev_rank) >= self.eps):
                new_df = df[prev_idx:i]
                new_df.reset_index(inplace=True)
                prev_idx = i
                bins_done += 1
                self.solve_bin(new_df,col)

            if (bins_done >= total_bins - 1):
                new_df = df[prev_idx:]
                new_df.reset_index(inplace=True)
                self.solve_bin(new_df,col)
                break
                


    def solve_bin(self, df, col):
        for r in range(0, len(df)):
            split = df.loc[r,'Feature']

            lhs = df['Feature'] <= df.loc[r, 'Feature']
            lhs_indices = df.loc[lhs, 'id'].tolist()
            
            rhs = df['Feature'] > df.loc[r, 'Feature']
            rhs_indices = df.loc[rhs, 'id'].tolist()
            
            
            if (len(lhs_indices) < self.min_leaf or len(rhs_indices) < self.min_leaf or
                    self.hessian[self.idxs][lhs_indices].sum() < self.min_child_weight or
                    self.hessian[self.idxs][rhs_indices].sum() < self.min_child_weight):
                continue

            poss_score = self.gain(lhs_indices, rhs_indices)

            if poss_score > self.score:
                self.col = col
                self.score = poss_score
                self.split = split

                    
    
    def gain(self,lhs,rhs):
        # Calculates the gain for a particular split.
        gradient = self.gradient[self.idxs]        
        hessian = self.hessian[self.idxs]        
        
        lhs_gradient = gradient[lhs].sum()
        lhs_hessian = hessian[lhs].sum()
        
        rhs_gradient = gradient[rhs].sum()
        rhs_hessian = hessian[rhs].sum()
        
        gain = 0.5*((lhs_gradient**2)/(lhs_hessian+self.lambda_) + (rhs_gradient**2)/(rhs_hessian+self.lambda_)-
                    ((gradient.sum())**2)/(hessian.sum()+self.lambda_)) - self.gamma
        
        return gain
    
    def split_col(self):    # Return values of a particular column
        return self.x.values[self.idxs, self.col]
    
    def is_leaf(self):      # Check if node is leaf or not
        # print(str(self.score) + " " +str(self.depth))
        return self.score == float('-inf') or self.depth <=0 or len(self.x) == 1
    
    def predict(self,x):
        return np.array([self.predict_val(xi) for xi in x])
    
    def predict_val(self,xi):
        # print(xi)
        if (self.is_leaf()):
            return self.val
        
        node = self.lhs if xi[self.col] <= self.split else self.rhs
        
        return node.predict_val(xi)
    
    
    

In [2]:
class XGBoostTree:
    def fit(self,x,gradient,hessian,subsample_cols = 1, min_leaf = 1, min_child_weight = 1, 
            depth = 7, lambda_ = 1, gamma = 1, eps = 0.01):
        self.root = Node(x, gradient, hessian, np.array(np.arange(len(x))), subsample_cols, min_leaf, min_child_weight,
                         depth, lambda_, gamma, eps)
    def predict(self,X):
        return self.root.predict(X.values)

In [3]:
class XGBoostRegression:
    
    def __init__(self):
        self.trees = []
        self.learning_rates = []
    
    def gradient(self,prediction, actual):
        return (2*(prediction-actual))
    
    def hessian(self,prediction, actual):
        return (np.full((prediction.shape[0],),2).astype('float'))
    
    def fit(self, X, Y, subsample_cols = 1, min_leaf = 1, min_child_weight = 1, 
            depth = 5, lambda_ = 1, gamma = 1, eps = 0.01, learning_rate = 0.4, boosting_rounds = 5):
        self.X = X
        self.Y = Y
        self.subsample_cols = subsample_cols
        self.min_leaf = min_leaf
        self.min_child_weight = min_child_weight
        self.depth = depth
        self.lambda_ = lambda_
        self.gamma = gamma
        self.eps = eps
        self.learning_rate = learning_rate
        self.boosting_rounds = boosting_rounds
        
        self.prediction = np.full((X.shape[0],1),np.mean(Y)).flatten().astype('float')
        
        # print(self.prediction)
        
        for booster in range(0,self.boosting_rounds):
            Gradient = self.gradient(np.array(self.prediction), np.array(self.Y))
            Hessian = self.hessian(np.array(self.prediction), np.array(self.Y))
            boosting_tree = XGBoostTree()
            boosting_tree.fit(  self.X, Gradient,Hessian, self.subsample_cols, self.min_leaf,
                                self.min_child_weight,self.depth,self.lambda_, self.gamma,
                                self.eps)
            
            new_val = boosting_tree.predict(self.X)
            new_learning_rate = self.Adam(self.Y, self.prediction, new_val)
            self.prediction = self.prediction + new_learning_rate*(boosting_tree.predict(self.X))
            self.trees.append(boosting_tree)
            self.learning_rates.append(new_learning_rate)
            print("Iteration: "+ str(booster) + " RMSE Score = " +str(np.sqrt(np.mean((self.predict(self.X) - self.Y)**2))))
    
    def predict(self,X):
        pred = np.zeros(X.shape[0])
        for i in range(len(self.trees)):
            pred += self.learning_rates[i]*self.trees[i].predict(X)
        return np.full((X.shape[0],),np.mean(self.Y)).astype('float') + pred
    
    
    def AdamLoss(self, old_res, alpha, new_val):
        return -2*np.sum(new_val*(old_res-alpha*new_val))
    
    def Adam(self, Y, base_pred, new_val):
        old_res = Y - base_pred
        print(old_res)
        loss = 0
        beta1 = 0.9
        beta2 = 0.99
        epsilon = 1e-18
        eta = 0.001
        m_old = 0
        v_old = 0
        mt = 0
        vt = 0
        mt_hat = 0
        vt_hat = 0
        t = 0
        converged = False
        alpha_old = self.learning_rate
        alpha = self.learning_rate
        while not converged:
            t = t+1
            loss = self.AdamLoss(old_res=old_res, alpha=alpha, new_val=new_val)
            mt = beta1*m_old + (1-beta1)*loss
            vt = beta2*v_old + (1-beta2)*loss
            mt_hat = mt/(1-(beta1 ** t))
            vt_hat = vt/(1-(beta2 ** t))
            alpha = alpha_old - (eta*mt_hat)/(math.sqrt(max(vt_hat,epsilon)) + epsilon)
            
            if alpha_old == alpha:
                converged = True 
            
            alpha_old = alpha
            m_old = mt
            v_old = vt
            
        return alpha

In [4]:
data = {'Dosage':[10,20,25,37],
        'Level' : [10,20,25,37]}
df = pd.DataFrame(data)

test_data = {'Dosage':[10,20,25,37],
             'Level' : [10,20,25,37]}
test_df = pd.DataFrame(test_data)

Y = [-10,7,8,-7]
model = XGBoostRegression()
model.fit(df,Y,boosting_rounds = 9,gamma = 0)

[-9.5  7.5  8.5 -6.5]
Iteration: 0 RMSE Score = 2.2082846689970383e-13
[ 2.59348099e-13 -2.05169215e-13 -2.32702746e-13  1.77635684e-13]
Iteration: 1 RMSE Score = 1.6185208837122475e-13
[ 1.90070182e-13 -1.50102153e-13 -1.70530257e-13  1.30562228e-13]
Iteration: 2 RMSE Score = 1.1831123320395927e-13
[ 1.38555833e-13 -1.10134124e-13 -1.24344979e-13  9.59232693e-14]
Iteration: 3 RMSE Score = 8.648003129016434e-14
[ 1.01252340e-13 -8.08242362e-14 -9.05941988e-14  7.01660952e-14]
Iteration: 4 RMSE Score = 6.340530252457267e-14
[ 7.46069873e-14 -5.95079541e-14 -6.57252031e-14  5.15143483e-14]
Iteration: 5 RMSE Score = 4.6597635579724805e-14
[ 5.50670620e-14 -4.35207426e-14 -4.79616347e-14  3.81916720e-14]
Iteration: 6 RMSE Score = 3.450204495357495e-14
[ 4.08562073e-14 -3.19744231e-14 -3.55271368e-14  2.84217094e-14]
Iteration: 7 RMSE Score = 2.5359788109947406e-14
[ 3.01980663e-14 -2.30926389e-14 -2.66453526e-14  2.04281037e-14]
Iteration: 8 RMSE Score = 1.8364041956551464e-14


In [5]:
model.predict(test_df)

array([-10.,   7.,   8.,  -7.])

In [6]:
with open('dataframe.pickle', 'rb') as f:
    new_df = pickle.load(f)

In [7]:
new_df.columns

Index(['Datetime', 'PJME_MW', 'Lag1', 'Lag2', 'Hour_sin', 'Hour_cos',
       'Day_sin', 'Day_cos'],
      dtype='object')

In [8]:
X = new_df.drop(columns=['PJME_MW','Datetime'])
Y = new_df['PJME_MW']

In [9]:
X


,Lag1,Lag2,Hour_sin,Hour_cos,Day_sin,Day_cos
0,27077.0,25591.0,0.258819,0.965926,-2.449294e-16,1.000000
1,25957.0,24235.0,0.500000,0.866025,-2.449294e-16,1.000000
2,24930.0,23121.0,0.707107,0.707107,-2.449294e-16,1.000000
3,24359.0,22445.0,0.866025,0.500000,-2.449294e-16,1.000000
4,24400.0,22332.0,0.965926,0.258819,-2.449294e-16,1.000000
...,...,...,...,...,...,...
127745,31448.0,32878.0,-0.866025,0.500000,1.716633e-02,0.999853
127746,31246.0,32586.0,-0.707107,0.707107,1.716633e-02,0.999853
127747,30526.0,31877.0,-0.500000,0.866025,1.716633e-02,0.999853
127748,29209.0,30590.0,-0.258819,0.965926,1.716633e-02,0.999853


In [10]:
Y

0         27160.0
1         25791.0
2         25052.0
3         24797.0
4         25026.0
           ...   
127745    44284.0
127746    43751.0
127747    42402.0
127748    40164.0
127749    38608.0
Name: PJME_MW, Length: 127750, dtype: float64

In [11]:
model = XGBoostRegression()
model.fit(X,Y,min_leaf = 5,min_child_weight = 5,boosting_rounds = 6)

0         -4988.386935
1         -6357.386935
2         -7096.386935
3         -7351.386935
4         -7122.386935
              ...     
127745    12135.613065
127746    11602.613065
127747    10253.613065
127748     8015.613065
127749     6459.613065
Name: PJME_MW, Length: 127750, dtype: float64
Iteration: 0 RMSE Score = 4124.297620510072
0          -757.185879
1          1194.646155
2           455.646155
3           200.646155
4           429.646155
              ...     
127745    12578.189534
127746    12045.189534
127747    10696.189534
127748    10321.093333
127749     8765.093333
Name: PJME_MW, Length: 127750, dtype: float64
Iteration: 1 RMSE Score = 3990.5102490982986
0         -1480.210149
1           471.621885
2          -267.378115
3          -522.378115
4          -293.378115
              ...     
127745    11407.297355
127746    12040.914712
127747    10691.914712
127748     9598.069062
127749     8042.069062
Name: PJME_MW, Length: 127750, dtype: float64
Iteration: 2 R